# Chain

**Chain**(체인)은 여러 컴포넌트(요소)를 정해진 순서대로 연결하여 **복잡한 AI 작업을 단계별로 자동화**할 수 있도록 돕는 구조이다.

- 각 컴포넌트는 입력을 받아 특정 처리를 수행한 후 다음 단계로 결과를 전달한다.
- 복잡한 작업을 여러 개의 단순한 단계로 나누고, 각 단계를 순차적으로 실행함으로써 전체 작업을 체계적으로 구성할 수 있다.

## 기본 개념

- 체인은 하나의 LLM 호출에 그치지 않고 **여러 LLM 호출이나 도구 실행을 순차적으로 연결**할 수 있다.
- 예를 들어, 사용자의 질문 → 검색 → 요약 → 응답 생성 같은 일련의 작업을 체인으로 구성할 수 있다.
- 체인을 사용하면 코드의 재사용성과 유지 보수성이 향상된다.

## LangChain에서의 Chain 구성 방식

LangChain은 다음 두 가지 방식을 통해 체인을 구성할 수 있다.

### 1. Off-the-shelf Chains 방식 (클래식 방식)

- LangChain에서 제공하는 **미리 정의된 Chain 클래스**(예: `LLMChain`, `SequentialChain`, `SimpleSequentialChain`)를 활용하는 방식이다.
- 이 방식은 LangChain의 **초기 구조**이며, 대부분의 클래스는 현재 **더 이상 사용되지 않음(deprecated)** 상태이다.

> 현재 LangChain에서는 이 방식을 권장하지 않는다.

### 2. LCEL (LangChain Expression Language) 방식

- 체인을 함수형 방식으로 선언할 수 있는 **표현식 기반의 체인 구성 언어**이다.
- LCEL 방식은 간결하고 선언적인 문법을 제공하여 **직관적이고 확장성 있는 체인 구성**이 가능하다.
- `Runnable`이라는 공통 인터페이스를 기반으로 다양한 요소를 조합하여 체인을 구성한다.
- 체인의 각 구성 요소는 `invoke()` 메서드로 실행된다.

In [1]:
# off-the-shelf 방식
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

prompt_template = PromptTemplate(
    template="{item}에 어울리는 이름 {count}개를 만들어 주세요."
)
model = ChatOpenAI(model="gpt-4o-mini")
parser = StrOutputParser()

In [2]:
from langchain import LLMChain

chain = LLMChain(
    prompt=prompt_template,
    llm=model,
    output_parser=parser
)
response = chain.invoke({"item": "가방", "count": 5})

C:\Users\Playdata\AppData\Local\Temp\ipykernel_16644\3018763423.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(


In [3]:
response

{'item': '가방',
 'count': 5,
 'text': '물론입니다! 가방에 어울리는 이름을 5개 제안드립니다.\n\n1. **세련된 소풍 (Chic Picnic)** - 가벼운 나들이나 소풍에 잘 어울리는 가방.\n2. **일상의 동반자 (Everyday Companion)** - 매일 사용할 수 있는 실용적인 디자인의 가방.\n3. **스타일리시 셔틀 (Stylish Shuttle)** - 도시에서의 이동을 쉽게 해주는 세련된 가방.\n4. **편안한 여행자 (Comfort Traveler)** - 여행 시 편리하게 사용할 수 있는 공간과 기능을 갖춘 가방.\n5. **심플 라인 (Simple Line)** - 미니멀한 디자인으로 어떤 옷과도 잘 어울리는 가방.\n\n이 이름들이 도움이 되길 바랍니다!'}

In [6]:
type(response)

dict

In [4]:
print(response['text'])

물론입니다! 가방에 어울리는 이름을 5개 제안드립니다.

1. **세련된 소풍 (Chic Picnic)** - 가벼운 나들이나 소풍에 잘 어울리는 가방.
2. **일상의 동반자 (Everyday Companion)** - 매일 사용할 수 있는 실용적인 디자인의 가방.
3. **스타일리시 셔틀 (Stylish Shuttle)** - 도시에서의 이동을 쉽게 해주는 세련된 가방.
4. **편안한 여행자 (Comfort Traveler)** - 여행 시 편리하게 사용할 수 있는 공간과 기능을 갖춘 가방.
5. **심플 라인 (Simple Line)** - 미니멀한 디자인으로 어떤 옷과도 잘 어울리는 가방.

이 이름들이 도움이 되길 바랍니다!


# [LCEL](https://python.langchain.com/docs/how_to/#langchain-expression-language-lcel) (LangChain Expression Language)
- LCEL은 LangChain의 핵심 기능인 체인(Chain)을 더욱 간결하고 유연하게 구성할 수 있도록 고안된 **선언형 체인(chain) 구성 언어**이다.
- 파이프 연산자 `|`를 사용해 선언적 방법으로 여러 작업을 연결한다.
- 체인을 구성하는 각 요소는 `Runnable` 타입으로, 체인 내에서 실행 가능한 단위이다.
- 각 단계는 invoke() 메서드를 통해 실행되며, 앞 단계의 출력이 다음 단계의 입력으로 자동 전달된다.
    - [Runnable 컴포넌트별 입출력 타입](https://python.langchain.com/docs/concepts/runnables/#input-and-output-types)
    - 각 컴포넌트의 input과 output 타입에 맞춰 값이 전달되도록 한다.
- https://python.langchain.com/v0.2/docs/concepts/#langchain-expression-language-lcel

## [Runnable](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.base.Runnable.html)
- LangChain의 Runnable은 실행 가능한 작업 단위를 캡슐화한 개념으로, 데이터 흐름의 각 단계를 정의하고 **체인(chain) 에 포함 되어**  복잡한 작업의 각 단계를 수행 한다.
- Chain을 구성하는 class들은 Runnable의 상속 받아 구현한다.
- **Prompt Template클래스**, **Chat 모델, LLM 모델 클래스**, **Output Parser 클래스** 등 다양한 컴포넌트가 Runnable을 상속받아 구현된다.

### 주요 특징
- 작업 단위의 캡슐화:
    - Runnable은 특정 작업(예: 프롬프트 생성, LLM 호출, 출력 파싱 등)을 수행하는 독립적인 컴포넌트이다.
    - 각 컴포넌트는 독립적으로 테스트 및 재사용이 가능하며, 조합하여 복잡한 체인을 구성할 수 있다.
- 체인 연결 및 작업 흐름 관리:
    - Runnable은 체인(chain, 일련의 연결된 작업 흐름)을 구성하는 기본 단위로 사용된다.
    - LangChain Expression Language(LCEL)를 사용하면 | 연산자를 통해 여러 Runnable을 쉽게 연결할 수 있다.
    - 력과 출력의 형식을 일관되게 유지하여 각 단계가 자연스럽게 연결된다.
- 모듈화 및 디버깅 용이성:
    - 각 단계가 명확히 분리되어 문제 발생 시 어느 단계에서 오류가 발생했는지 쉽게 확인할 수 있다.
    - 복잡한 작업을 작은 단위로 나누어 체계적으로 관리할 수 있다.
      
### Runnable의 표준 메소드
- 모든 Runnable이 구현하는 공통 메소드
    - `invoke()`: 단일 입력을 처리하여 결과를 반환.
    - `batch()`: 여러 입력 데이터들을 한 번에 처리.
    - `stream()`: 입력에 대해 스트리밍 방식으로 응답을 반환.
    - `ainvoke()`: 비동기 방식으로 입력을 처리하여 결과를 반환.

### Runnable의 주요 구현체(하위 클래스)

- `RunnableSequence`
    - 여러 `Runnable`을 순차적으로 연결하여 실행하는 구성이다.
    - 각 단계의 출력이 다음 단계의 입력으로 전달된다.
    - LCEL을 사용하여 체인을 구성할 경우 자동으로 `RunnableSequence`로 변환된다.
-  `RunnablePassThrough`
    - 입력 데이터를 가공하지 않고 그대로 다음 단계로 전달하는 `Runnable`이다.
    - 선택적으로 미리 정의된 키-값 쌍을 함께 전달할 수 있다.

- `RunnableParallel`
    - 여러 `Runnable`을 병렬로 실행한 후, 결과를 결합하여 다음 단계로 전달한다.
    - 병렬 처리를 통해 처리 속도를 개선할 수 있다.

- `RunnableLambda`
    - 일반 함수 또는 `lambda` 함수를 `Runnable`로 변환하여 체인에 포함할 수 있다.
    - 사용자 정의 함수로 동작을 확장할 때 유용하다.

#### Runnable 예제

In [11]:
from langchain_core.runnables import Runnable # 모든 Runnable의 최상위

# 사용자 정의 Runnable 클래스를 만들어보자.
class MyRunnable(Runnable):
    # invoke(): 구현하는 Runnable이 해야하는 작업을 구현하는 메소드
    def invoke(self, input_data:str, config:dict=None):
        # input_data: 입력 값
        # config: 일 할 때 필요한 설정 값들
        if config is not None and config.get('lang') == "en":
            return f"Explain {input_data} is one sentence."
        return f"{input_data}에 대해 한 문장으로 설명해줘."

In [14]:
my_runnable = MyRunnable()
my_runnable.invoke("사과")

'사과에 대해 한 문장으로 설명해줘.'

In [15]:
my_runnable.invoke("Apple", config={"lang": "en"})

'Explain Apple is one sentence.'

In [17]:
from langchain_openai import ChatOpenAI

my_runnable = MyRunnable()
model = ChatOpenAI(model='gpt-4o-mini')

# prompt = my_runnable.invoke("Apple", config={"lang": "en"})
prompt = my_runnable.invoke("사과")
response = model.invoke(prompt)
print(response)

content='사과는 달콤하고 상큼한 맛을 지닌 과일로, 다양한 품종이 있으며 건강에 유익한 영양소가 풍부하다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 19, 'total_tokens': 55, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BgjJKpRjwGPfvVsWgm4NUGAvCfn6R', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--ddfccbc6-11d9-4f0a-b682-b6199d91bac3-0' usage_metadata={'input_tokens': 19, 'output_tokens': 36, 'total_tokens': 55, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [19]:
# chain으로 묶어보자. Runnable | Runnable | Runnable
chain = my_runnable | model
# chain 호출도 invoke 메서드 사용
res = chain.invoke("과일 배")
print(res.content)

과일 배는 주로 아시아에서 재배되는 과일로, 부드럽고 달콤한 맛과 아삭한 식감을 가진 과일입니다.


In [20]:
# 기본 체인 구성: prompt_template -> model -> output parser
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import CommaSeparatedListOutputParser

# role: system(지침 지정), user/human, ai/assistant
prompt_template = ChatPromptTemplate(
    messages=[
        ("system", "당신은 오랜 경력의 한국 관광 가이드입니다. 여행객들에게 설명하듯이 친절하게 답변을 해주세요."),
        ("human", "{query}")
    ]
)
model = ChatOpenAI(model='gpt-4o-mini', temperature=1.0)

chain = prompt_template | model | StrOutputParser()

print(type(chain))

<class 'langchain_core.runnables.base.RunnableSequence'>


In [21]:
query = "서울에서 꼭 가봐야하는 여행지를 세 곳만 알려줘."
response = chain.invoke({"query": query})

In [22]:
print(response)

안녕하세요! 서울 여행을 계획하고 계시군요. 서울은 역사와 현대가 조화를 이루는 멋진 도시입니다. 꼭 가봐야 할 여행지를 세 곳 소개해 드릴게요.

1. **경복궁**: 서울의 대표적인 고궁으로, 조선왕조의 주요 거처였습니다. 아름다운 건축물과 넓은 정원을 구경할 수 있으며, 매일 열리는 수문장 교대식도 꼭 보시길 추천드립니다. 경복궁 근처에는 국립고궁박물관도 있어 궁의 역사와 문화를 더 깊이 이해할 수 있습니다.

2. **명동**: 한국의 대표적인 쇼핑과 음식의 중심지인 명동은 활기차고 다양한 매력을 가진 곳입니다. 다양한 패션 매장과 화장품 가게, 길거리 음식도 즐길 수 있어 젊은층에게 특히 인기가 많습니다. 또한, 명동성당도 꼭 들러보세요. 아름다운 고딕 양식의 건축물이 인상적입니다.

3. **홍대**: 홍익대학교 주변 지역으로, 예술과 문화의 중심지입니다. 다양한 카페와 갤러리, 독립 음악 공연도 많이 열려 젊은 감각을 느낄 수 있습니다. 길거리 공연과 플리 마켓도 종종 열리니 우연히 마주칠 수 있는 특별한 체험도 기대해보세요.

서울은 정말 다양한 매력을 가진 도시이니, 이렇게 세 곳을 방문하면서 즐거운 추억 많이 만드시길 바랍니다! 여행 중 궁금한 점이 있으면 언제든지 물어보세요.


In [23]:
while True:
    query = input("질문:")
    if query == "!quit":
        break
    resp = chain.invoke({"query": query})
    print("User:", query)
    print("AI:", resp)
    print("-"*50)

User: 
AI: 안녕하세요! 궁금한 점이 있으신가요? 한국의 아름다운 명소, 문화, 음식 등 어떤 것이든 편하게 질문해 주세요. 제가 기쁜 마음으로 도와드리겠습니다!
--------------------------------------------------
User: 경복궁에 대해 설명해줘.
AI: 안녕하세요! 경복궁에 대해 말씀드리겠습니다. 경복궁은 서울특별시에 위치한 조선왕조의 가장 큰 궁궐 중 하나로, 1395년에 건립되었습니다. 이 궁궐은 조선의 태조 이성계에 의해 세워졌으며, 조선 왕조의 정치와 문화의 중심지로서 중요한 역할을 했습니다.

경복궁의 이름은 '경복'(慶福)이란 한자로 '복을 기뻐하다'는 의미를 가지고 있습니다. 이 궁궐은 아름다운 전통 건축과 넓은 정원, 그리고 궁궐의 상징인 근정전을 비롯한 여러 전각들로 유명합니다. 근정전은 조선의 왕이 공식적으로 접견을 하고 국가의 중요한 의사결정을 내리는 장소였습니다.

특히, 경복궁의 정문인 광화문은 매혹적인 인상과 함께 궁궐의 시작을 알리는 상징적인 문입니다. 광화문에서는 매일 오전 10시와 오후 2시에 변경식도 열리며, 전통 복장을 입은 군인들이 경비를 서고 있어 관광객들에게 큰 인기를 끌고 있습니다.

궁궐 내부에는 국립고궁박물관과 국립민속박물관이 있어 조선의 역사와 문화에 대해 보다 깊이 이해할 수 있는 기회를 제공합니다.

경복궁은 사계절마다 그 매력이 다르니, 봄에는 아름다운 벚꽃과 가을의 단풍이 특히 아름답습니다. 방문하시게 되면 경복궁의 역사와 아름다움을 느끼며, 조선 왕조가 남긴 유산을 충분히 만끽하실 수 있을 것입니다.

여행 준비 잘 하시고, 경복궁에서 좋은 시간 보내시길 바랍니다!
--------------------------------------------------


#### RunnableLambda 예제

In [24]:
from langchain_core.runnables import RunnableLambda

# RunnableLambda(함수) -> 함수를 실행하는 Runnable을 생성한다.
my_runnable2 = RunnableLambda(lambda input_data: f"{input_data}를 한 문장으로 설명해줘.")
my_runnable2.invoke("LLM")

'LLM를 한 문장으로 설명해줘.'

In [ ]:
chain = my_runnable2 | model
chain.invoke("LLM")
# invoke(입력데이터:str|dict, 설정정보:dict)
## 입력 데이터가 여러개일 경우, dict등의 자료구조를 이용해서 받는다.

AIMessage(content='LLM(대규모 언어 모델)은 자연어 처리를 위해 대량의 텍스트 데이터를 학습하여 사람의 언어를 이해하고 생성하는 인공지능 모델입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 18, 'total_tokens': 58, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BgkAE8J3OGKUfcSNVtIi1wYl9QoUW', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--bc9e09c7-e826-4c7c-957f-ac96baf41b9f-0', usage_metadata={'input_tokens': 18, 'output_tokens': 40, 'total_tokens': 58, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

#### RunnablePassthrough 예제

In [ ]:
# 1. 앞 Runnable이 처리한 결과를 다음 Runnable에 그대로 전달.
from langchain_core.runnables import RunnablePassthrough

# RunnablePassthrough().invoke("안녕하세요") 
RunnablePassthrough().invoke({"key": "value"}) # 받은 것을 그냥 그대로 넘긴다.

{'key': 'value'}

In [ ]:
# 2. 앞 Runnable이 처리한 결과에 Item을 추가해서 다음 Runnable에 전달.
# RunnablePassthrough().assign(key1=Runnable, key2=Runnable, ...)
# 받은 dictionary에 key:Runnable반환값 추가해서 다음으로 전달.
address_runnable = RunnableLambda(lambda x: "서울시 금천구")
phone_runnable = RunnableLambda(lambda x: "010-1111-2222")

RunnablePassthrough.assign(address=address_runnable, phone=phone_runnable).invoke({"name": "홍길동"})

{'name': '홍길동', 'address': '서울시 금천구', 'phone': '010-1111-2222'}

#### RunnableSequence 예제

In [ ]:
from langchain_core.runnables import RunnableSequence

run1 = RunnableLambda(lambda x: x + 1)
run2 = RunnableLambda(lambda x: x * 2)

chain = run1 | run2
# chain = RunnableSequence(run1, run2) # 동일한 기능
print(type(chain))
chain.invoke(30)

<class 'langchain_core.runnables.base.RunnableSequence'>


62

#### RunnableParallel 예제

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnableLambda

run1 = RunnableLambda(lambda x: x + 1)
run2 = RunnableLambda(lambda x: x * 2)
run3 = RunnableLambda(lambda x: x // 3)

runnable = RunnableParallel(
    {
        "result1": run1,
        "result2": run2,
        "result3": run3,
        "result4": RunnablePassthrough() # 전달받은 값을 그대로 전달하는 용도
    }
)
# 각 Runnable들을 각각 실행하고, 그 결과를 key에 할당한 dictionary를 반환한다. (순서대로 실행하지 않음.)

runnable.invoke(20)

{'result1': 21, 'result2': 40, 'result3': 6, 'result4': 20}

#### LCEL Chain 예제

In [ ]:
# 음식 이름을 받아서 레시피를 영어로 출력하는 chain을 구성해보자.
# 흐름: prompt template -> model -> output parser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from textwrap import dedent

prompt_template = PromptTemplate(
    template=dedent("""
    # Instruction
    당신은 숙련된 요리 연구가입니다. 요청한 음식의 레시피를 작성해 주세요.
                    
    # Input data
    음식 이름: {food}
    """)
)
model = ChatOpenAI(model='gpt-4o-mini')

food_chain = prompt_template | model | StrOutputParser()

In [38]:
response = food_chain.invoke({"food": "pasta"})

In [39]:
print(response)

# Pasta Recipe

## Ingredients

- 400g spaghetti or your preferred pasta
- 2 tablespoons olive oil
- 3 cloves garlic, minced
- 1 can (400g) diced tomatoes
- 1 teaspoon dried oregano
- 1 teaspoon dried basil
- Salt and pepper to taste
- Fresh basil leaves for garnish (optional)
- Grated Parmesan cheese for serving (optional)

## Instructions

1. **Boil the Pasta**: 
   - In a large pot, bring water to a boil. Add a generous pinch of salt.
   - Add the spaghetti (or your chosen pasta) and cook according to the package instructions until al dente. 

2. **Prepare the Sauce**: 
   - While the pasta is cooking, heat olive oil in a large skillet over medium heat.
   - Add the minced garlic and sauté for about 1 to 2 minutes, or until fragrant, being careful not to burn it.

3. **Add Tomatoes**: 
   - Pour the canned diced tomatoes into the skillet with the garlic.
   - Stir in the dried oregano and dried basil. Season with salt and pepper to taste.
   - Let the sauce simmer for about 10-15 mi

In [62]:
# 번역 chain (입력 내용을 특정 언어로 번역한다.)
# 흐름: prompt template -> model -> output parser

prompt_template = PromptTemplate(
    template=dedent("""
    # Instruction
    당신은 다국어가능한 숙력된 번역가입니다. 
    요청된 문장을 {language} 로 번역해 주세요.
                    
    # Input data(번역할 문장)
    {content}
                    
    # Output Indicator
    - 번역한 내용만 출력해 주세요.
    """)
)

translate_chain = prompt_template | model | StrOutputParser()

In [ ]:
response_german = translate_chain.invoke({"content": response, "language": "독일어"})

In [ ]:
print(response_german)

# Pasta-Rezept

## Zutaten

- Spaghetti oder gewünschte Pasta 400 g
- Olivenöl 2 Esslöffel
- Gehackter Knoblauch 3 Zehen
- Gehackte Tomaten aus der Dose 1 Dose (400 g)
- Getrockneter Oregano 1 Teelöffel
- Getrockneter Basilikum 1 Teelöffel
- Salz und Pfeffer (nach Geschmack)
- Frische Basilikumblätter zur Dekoration (optional)
- Geriebener Parmesan zum Servieren (optional)

## Zubereitung

1. **Pasta kochen**:
   - Einen großen Topf mit Wasser zum Kochen bringen. Ausreichend Salz hinzufügen.
   - Spaghetti (oder die gewählte Pasta) hineingeben und nach Packungsanweisung al dente kochen.

2. **Soße vorbereiten**:
   - Während die Pasta kocht, in einer großen Pfanne das Olivenöl bei mittlerer Hitze erhitzen.
   - Den gehackten Knoblauch hinzufügen und etwa 1–2 Minuten anbraten, bis er duftet. Dabei darauf achten, dass er nicht verbrennt.

3. **Tomaten hinzufügen**:
   - Die gehackten Tomaten aus der Dose in die Pfanne mit dem Knoblauch gießen.
   - Getrockneten Oregano und Basilikum hinz

## Chain과 Chain간의 연결

In [63]:
# food_chain + translate_chain
## food_chain_prompt의 변수: food
## translate_chain_prompt의 변수: language, content


chain = {
    "content": food_chain,
    "language": RunnableLambda(lambda x: x['language'])
} | translate_chain
# 병렬로 처리한 첫번째 체인 결과인 {"content":레시피, "language":언어}를 두번째 체인(transalte_chain)에 넘긴다.

In [66]:
response = chain.invoke({"food": "파스타", "language": "일본어"})

In [67]:
print(response)

パスタレシピ

### 材料
- スパゲッティ: 200g
- オリーブオイル: 大さじ2
- 刻みニンニク: 3片
- チェリートマト: 200g（半分に切ったもの）
- バジルの葉: 一握り（新鮮なもの）
- 塩: 適量
- 胡椒: 適量
- パルメザンチーズ: 適量（オプショナル）

### 調理方法

1. **パスタを茹でる**:
   - 大きな鍋に水を沸かし、塩を加えます。
   - 沸騰した水にスパゲッティを入れ、パッケージに記載された時間（通常8〜10分）に従って茹でます。
   - パスタが茹で上がったら、ザルにあけて水を切り、少々のオリーブオイルを加えてよく混ぜます。

2. **ソースを作る**:
   - フライパンにオリーブオイルを入れ、中火で加熱します。
   - 刻みニンニクを加え、茶色がかるまで炒めます（約1〜2分）。
   - チェリートマトを追加し、塩と胡椒で味付けしながら約5分間調理します。チェリートマトが柔らかくなり、汁が出るまで炒めます。

3. **パスタとソースを合わせる**:
   - 茹でたパスタをフライパンに加え、新鮮なバジルの葉を入れます。
   - すべての材料をよく混ぜながら1〜2分間さらに加熱します（希望する場合はここでパルメザンチーズを加えても良いです）。

4. **サーブする**:
   - 完成したパスタを皿に盛り付け、バジルの葉とパルメザンチーズを添えて飾ります。
   - お好みでさらに胡椒を振りかけて提供します。

### ヒント
- 様々な材料を加えてアレンジできます。（例：海鮮、野菜、肉など）
- ソースに刻んだ唐辛子を入れると辛みを加えることができます。
- ソースにクリームを加えてクリームソースパスタにアレンジできます。

美味しく召し上がれ！


# 사용자 함수를 Chain에 적용하기

## 사용자 함수를 Runnable로 정의 (RunnableLambda)
- 임의의 함수를 Runnable로 정의 할 수있다.
  - chain에 포함할 기능을 함수로 정의할 때 주로 사용. 
- `RunnableLambda(함수)` 사용
  - 함수는 invoke() 메소드를 통해 입력받은 값을 받을 **한개의 파라미터**를 선언해야 한다.

## 사용자 함수를 Chain으로 정의
- Chain 을 구성하는 작업 사이에 추가 작업이 필요할 경우, 중간 결과를 모두 사용해야 하는 경우 함수로 구현한다.
- `@chain` 데코레이터를 사용해 함수에 선언한다.

### Runnable 에 사용할 **사용자 정의 함수** 구문
- 이전 Chain의 출력을 입력 받는 **파라미터를 한개** 선언한다. (첫번째 파라미터)
- `invoke()`로 호출 할때 전달 하는 추가 설정을 입력받는 파라미터를 선언한다.(두번째 파라미터 - Optional)
  - RunnableConfig 타입의 값을 받는데 Dictionary 형식으로 `{"configuable": {"설정이름":"설정값"}}` 형식으로 받는다.
- 만약 함수가 여러개의 인자를 받는 경우 단일 입력을 받아들이고 이를 여러 인수로 풀어내는 래퍼 함수를 작성하여 Runnable로 만든다.
  ```python
  def plus(num1, num2):
      ...

  def wrapper_plus(nums:dict|list):
      return plus(nums['num1'], nums['num2'])
  ```

# Cache

- 응답 결과를 저장해서 같은 질문이 들어오면 LLM에 요청하지 않고 저장된 결과를 보여주도록 한다.
    - 처리속도와 비용을 절감할 수 있다.
    - 특히 chatbot같이 비슷한 질문을 하는 경우 유용하다.
- 저장 방식은 `메모리`, `sqlite` 등 다양한 방식을 지원한다.
  
    ```python
    set_llm_cache(Cache객체)
    ```